In [1]:
# Import Node and Workflow object and FSL interface
import numpy as np
from nipype import Node, Workflow, IdentityInterface
from os.path import abspath
import pycartool as cart
import mne
import pandas as pd

210309-17:24:02,781 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
spi_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cartool/sub-01/sub-01.spi'
rois_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cartool/sub-01/Rois/sub-01_label-L2008_desc-scale1.pickle.rois'
epochs_fif_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01_epo.fif'
xyz_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cartool/sub-01/sub-01.xyz'
is_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cartool/sub-01/sub-01.LAURA.is'
roi_ts_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cmp-v3.0.0-beta-RC1/sub-01/eeg/sub-01.L2008_desc-scale1_LAURA.npy'

svd_params =  {'toi_begin' : 120,
               'toi_end' : 500}

epochs_fname = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01_FACES_250HZ_prepd.set'

In [3]:
i0 = Node(IdentityInterface(fields=['epochs', 
                                    'behav', 
                                    'epochs_fif', 
                                    ], 
                            mandatory_inputs=True), name = 'input_node_0'
         )

i0.inputs.epochs = epochs_fname
i0.inputs.behav = behav_fname
i0.inputs.epochs_fif = epochs_fif_fname


i1 = Node(IdentityInterface(fields=['src_file', 
                                    'invsol_file', 
                                    'lamda', 
                                    'rois_file', 
                                    'svd_params',
                                    'roi_ts_file'], 
                            mandatory_inputs=True), name = 'input_node_1'
         )

i1.inputs.src_file = spi_fname
i1.inputs.invsol_file = is_fname
i1.inputs.lamda = 6
i1.inputs.rois_file = rois_fname
i1.inputs.svd_params = svd_params
i1.inputs.roi_ts_file = roi_ts_fname

In [4]:
from eeg_implementation.cmptklib.interfaces import invsol
from eeg_implementation.cmptklib.interfaces import eeglab2fif


In [5]:
eeglab2fif_node = Node(eeglab2fif.EEGLAB2fif(), name="eeglab2fif")
is_node = Node(invsol.CartoolInverseSolutionROIExtraction(), name="invsolution")

In [6]:
wf = Workflow(name="roi_ts", base_dir="./output/working_dir/")

wf.connect([(i0, eeglab2fif_node,[('epochs','eeg_ts_file'),
                          ('behav','behav_file'),
                          ('epochs_fif','epochs_fif_fname'),
                         ]
            )])

wf.connect([(i1, is_node,[('src_file','src_file'),
                          ('invsol_file','invsol_file'),
                          ('lamda','lamda'),
                          ('rois_file','rois_file'),
                          ('svd_params','svd_params'),
                          ('roi_ts_file','roi_ts_file'),
                         ]
            )])

wf.connect([(eeglab2fif_node, is_node,[('b','eeg_ts_file')]
            )])
wf.run()

210309-17:24:03,558 nipype.workflow INFO:
	 Workflow roi_ts settings: ['check', 'execution', 'logging', 'monitoring']
210309-17:24:03,563 nipype.workflow INFO:
	 Running serially.
210309-17:24:03,564 nipype.workflow INFO:
	 [Node] Setting-up "roi_ts.eeglab2fif" in "/Volumes/SSD-Bercows/Joan/research/2_projects/CMP_EEG/output/working_dir/roi_ts/eeglab2fif".
210309-17:24:03,567 nipype.workflow INFO:
	 [Node] Outdated cache found for "roi_ts.eeglab2fif".
210309-17:24:03,576 nipype.workflow INFO:
	 [Node] Running "eeglab2fif" ("eeg_implementation.cmptklib.interfaces.eeglab2fif.EEGLAB2fif")
Extracting parameters from /Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01_FACES_250HZ_prepd.set...


/Volumes/SSD-Bercows/Joan/research/2_projects/CMP_EEG/eeg_implementation/cmptklib/interfaces/eeglab2fif.py:48: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.read_epochs_eeglab(epochs_file, events=None, event_id=None, eog=(), verbose=None, uint16_codec=None)
/Volumes/SSD-Bercows/Joan/research/2_projects/CMP_EEG/eeg_implementation/cmptklib/interfaces/eeglab2fif.py:48: RuntimeWarning: Data file name in EEG.data (01_DNP_FACES_250HZ_dtle_icap_iav_z.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-01_FACES_250HZ_prepd.fdt).
  epochs = mne.read_epochs_eeglab(epochs_file, events=None, event_id=None, eog=(), verbose=None, uint16_codec=None)


588 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Overwriting existing file.
210309-17:24:23,292 nipype.workflow INFO:
	 [Node] Finished "roi_ts.eeglab2fif".
210309-17:24:23,293 nipype.workflow INFO:
	 [Node] Setting-up "roi_ts.invsolution" in "/Volumes/SSD-Bercows/Joan/research/2_projects/CMP_EEG/output/working_dir/roi_ts/invsolution".
210309-17:24:23,304 nipype.workflow INFO:
	 [Node] Running "invsolution" ("eeg_implementation.cmptklib.interfaces.invsol.CartoolInverseSolutionROIExtraction")
Reading /Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01_epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...     996.00 ms
        0 CTF compensation matrices available
588 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cartool/sub-01/su

In [77]:
from nipype.interfaces.io import BIDSDataGrabber
bg = Node(BIDSDataGrabber(index_derivatives=True), name='bids-grabber',anat_only=False)
bg.inputs.base_dir = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/'
bg.inputs.subject = '01'
bg.inputs.output_query = {'EEG': {'scope': 'EEGLAB',
                                  'suffix': 'prepd',
                                  'extensions': ['set']},
                          'behav': {'scope': 'EEGLAB',
                                  'suffix': 'behav',
                                  'extensions': ['txt']},
                         'rois_file': {'scope': 'Cartool',
                                  'suffix': 'scale3',
                                  'extensions': ['rois']},
                         'src_file': {'scope': 'Cartool',
                                  'extensions': ['spi']},
                         'invsol_file': {'scope': 'Cartool',
                                  'extensions': ['LAURA.is']},
                         'atlas': {'scope': 'Connectome Mapper',
                                   'suffix': 'atlas',
                                   'extension': 'nii.gz',
                                   'parcellation_name': 'L2008_desc-scale4'}
                         }
result = bg.run()

210310-10:34:41,923 nipype.workflow INFO:
	 [Node] Setting-up "bids-grabber" in "/private/var/folders/t7/t6fr54mx5cl1t4t9fcmq_rzr0000gq/T/tmp1_wxsmri/bids-grabber".
210310-10:34:41,928 nipype.workflow INFO:
	 [Node] Running "bids-grabber" ("nipype.interfaces.io.BIDSDataGrabber")
210310-10:34:42,550 nipype.workflow INFO:
	 [Node] Finished "bids-grabber".


In [78]:
result.outputs.EEG

['/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01-MOTION_250HZ_prepd.set',
 '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/eeglab/sub-01/sub-01_FACES_250HZ_prepd.set']

96.0

In [1]:
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from traits.api import *
from cmp.pipelines.common import *
import datetime
from cmp.stages.eeg.eeg_preprocessing import EEGPreprocessingStage
from cmp.stages.eeg.eeg_inverse_solution import EEGInverseSolutionStage

210310-13:10:25,892 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


ModuleNotFoundError: No module named 'cmp.stages.eeg'

In [102]:
class Global_Configuration(HasTraits):

    def __init__(self,base_dir):
        self.base_directory = base_dir
    

class EEGPipeline(Pipeline):
    
    def __init__(self,base_dir,subject_id):
        self.global_conf = Global_Configuration(base_dir)
        self.subject_id = subject_id

        self.stages = {'EEGPreprocessing': EEGPreprocessingStage(bids_dir=project_info.base_directory,
                                                           output_dir=self.output_directory),
                       'EEGInverseSolution': EEGInverseSolutionStage(bids_dir=project_info.base_directory,
                                                           output_dir=self.output_directory),
                       }
        cmp_common.Pipeline.__init__(self, project_info)

        
self = EEGPipeline('/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/','sub-01')
cmp_deriv_subject_directory = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cmp-v3.0.0-beta-RC1/'

NameError: name 'EEGPreprocessing' is not defined

In [101]:
datasource = pe.Node(interface=nio.BIDSDataGrabber(index_derivatives=True), name='bids-grabber',anat_only=False)
datasource.inputs.base_dir = self.global_conf.base_directory
datasource.inputs.subject = self.subject_id
datasource.inputs.output_query = {
                                    'EEG': {
                                            'scope': 'EEGLAB',
                                            'suffix': 'prepd',
                                            'extensions': ['set']
                                            },
                                    'behav': {'scope': 'EEGLAB',
                                            'suffix': 'behav',
                                            'extensions': ['txt']},
                                    'rois_file': {
                                            'scope': 'Cartool',
                                            'suffix': 'scale3',
                                            'extensions': ['rois']
                                            },
                                    'src_file': {
                                            'scope': 'Cartool',
                                            'extensions': ['spi']
                                            },
                                    'invsol_file': {
                                            'scope': 'Cartool',
                                            'extensions': ['LAURA.is']
                                            },
                                    }
# Data sinker for output
sinker = pe.Node(nio.DataSink(), name="eeg_sinker")
sinker.inputs.base_directory = os.path.abspath(cmp_deriv_subject_directory)

 # Clear previous outputs
self.clear_stages_outputs()

# Create common_flow
eeg_flow = pe.Workflow(name='eeg_pipeline', base_dir=os.path.abspath(
    nipype_deriv_subject_directory))


eeg_inputnode = pe.Node(interface=util.IdentityInterface(
                    fields=['epochs', 
                            'behav', 
                            'src_file', 
                            'invsol_file', 
                            'lamda', 
                            'rois_file', 
                            'svd_params'
                            ], 
                    mandatory_inputs=True), name="inputnode")

eeg_outputnode = pe.Node(interface=util.IdentityInterface(
                    fields=["roi_ts_file"]), name="outputnode")

eeg_flow.add_nodes([eeg_inputnode, eeg_outputnode])

eeg_flow.connect([
    (datasource, eeg_inputnode, 
        [("EEG", "epochs"),
        ('behav','behav_file'),
        ('src_file','src_file')
        ('invsol_file','invsol_file'),            	
        ]),
])

eeg_flow.connect([(eeg_inputnode, eeglab2fif_node,
                 [('epochs','eeg_ts_file'),
                  ('behav','behav_file'),
                 ]
    )])

eeg_flow.connect([(eeg_inputnode, is_node,
                 [('src_file','src_file'),
                  ('invsol_file','invsol_file'),
                  ('lamda','lamda'),
                  ('rois_file','rois_file'),
                  ('svd_params','svd_params'),
                 ]
    )])

eeg_flow.connect([(eeglab2fif_node, is_node,
                 [('fif_ts_file','epochs_file'),

                 ]
    )])

eeg_flow.connect([(is_node, eeg_outputnode,
                 [('roi_ts_file','roi_ts_file'),
                 ]
    )])

AttributeError: 'EEGPipeline' object has no attribute 'stages'